# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [25]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
                        .appName("Reto 1")
                        .master("local")
                        .getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2bedc3f2


### Carga de CSV

In [2]:
val trade_details = spark.read.format("csv")
                              .option("header", "true")
                              .option("delimiter", ";")
                              .load("Desktop/Big Data/Retos Big Data/Spark for Data Engineers/reto1/trade_details.csv")

val trade_details_snapshot = spark.read.format("csv")
                                       .option("header", "true")
                                       .option("delimiter", ";")
                                       .load("Desktop/Big Data/Retos Big Data/Spark for Data Engineers/reto1/trade_details_snapshot.csv")

trade_details: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]
trade_details_snapshot: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.


In [112]:
var data = trade_details.where($"mfamily" === "CURR" && $"mgroup" === "FXD" && $"mtype" === "SWLEG")

val origin_contract_number = data.select("origin_contract_number", "origin_trade_number").distinct()
val list_OCN = origin_contract_number.map(r => r.getString(0)).collect.toList


def getLongLeg(ds: DataFrame, list: List[String]): List[String] ={
    var aux : String = ""
    var longLegOTN: List[String] = List()
    var l: List[Any] = List()
    
    for(aux <- list){
         if(!ds.where($"origin_contract_number" === aux && $"maturity" === "NULL").isEmpty){
             l = trade_details_snapshot.where(col("origin_contract_number") === aux).orderBy(desc("maturity")).select("origin_trade_number").take(1).toList
             longLegOTN = longLegOTN.:+(l(0).toString.substring(1,l(0).toString.length-1))
         }else{
             l = ds.where(col("origin_contract_number") === aux).orderBy(desc("maturity")).select("origin_trade_number").take(1).toList
             longLegOTN = longLegOTN.:+(l(0).toString.substring(1,l(0).toString.length-1))
         }
    }
    
    longLegOTN
}

var longLeg = getLongLeg(origin_contract_number, list_OCN)
var aux = data.collect.toList
var res: List[String] = List()

for(it <- aux){
   if(longLeg.contains(it(3))){
       res = res.:+(it.toString.substring(1,it.toString.length-1))
   }
}
aux = trade_details.where( $"mtype" =!= "SWLEG" || col("mtype").isNull).collect.toList

for(it <- aux){
       res = res.+:(it.toString.substring(1,it.toString.length-1))
 
}

val resultado = res

//not very efficient working with lists

data: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mfamily: string, mgroup: string ... 4 more fields]
origin_contract_number: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [origin_contract_number: string, origin_trade_number: string]
list_OCN: List[String] = List(18724280, 21622649, 19883451, 19622128, 19622128, 19883451, 21622649, 18622136, 18724280, 18622136)
getLongLeg: (ds: org.apache.spark.sql.DataFrame, list: List[String])List[String]
longLeg: List[String] = List(19772400, 22798005, 20980932, 20665178, 20665178, 20980932, 22798005, 19665186, 19772400, 19665186)
aux: List[org.apache.spark.sql.Row] = List([IRD,BOND,null,316391872,678876251,2021-09-22], [CURR,FXD,FXD,32734782,54853428,2021-09-22], [IRD,LN_BR,null,1111,2222,2022-10-06], [IRD,IRS,null,22222...


In [70]:
var td_snapshot = trade_details_snapshot.withColumnRenamed("origin_contract_number", "OCN_snapshot")
                                        .withColumnRenamed("origin_trade_number", "OTN_snapshot")
                                        .withColumnRenamed("maturity", "maturity_snapshot")
                                        .withColumnRenamed("mfamily", "mfamily_snapshot")
                                        .withColumnRenamed("mgroup", "mgroup_snapshot")
                                        .withColumnRenamed("mtype", "mtype_snapshot")

var data = trade_details.join(td_snapshot,
                              trade_details("origin_contract_number") === td_snapshot("OCN_snapshot")
                              && trade_details("origin_trade_number") === td_snapshot("OTN_snapshot"), "left")

data = data.drop(data("OTN_snapshot"))
           .drop(data("OCN_snapshot"))
           .drop(data("mfamily_snapshot"))
           .drop(data("mgroup_snapshot"))
           .drop(data("mtype_snapshot"))

//data.show(5, 25, true)

val window = Window.partitionBy("origin_contract_number")

data = data.withColumn("long_leg", max("maturity_snapshot").over(window))

data = data.where(col("mtype") =!= "SWLEG" || col("mtype").isNull ||  col("mtype") === "SWLEG" && col("maturity_snapshot") === col("long_leg"))
data.drop(data("maturity_snapshot")).drop(data("long_leg")).show()

var result = data

//more efficient

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    IRD|    CF| null|           20513130|              19433281|2021-07-06|
|    IRD|   IRS| null|          555111222|             555111222|      NULL|
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|
|    EQD| EQUIT|  FWD|           10000001|              10000001|2019-07-02|
|   CURR|   FXD|SWLEG|           20665178|              19622128|2020-12-30|
|    IRD|   IRS| null|           33333333|              33333333|2024-10-15|
|    IRD|   IRS| null|          444111222|             444111222|      NULL|
|    IRD|    CF| null|           20533916|              19453781|2023-07-06|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|

td_snapshot: org.apache.spark.sql.DataFrame = [mfamily_snapshot: string, mgroup_snapshot: string ... 4 more fields]
data: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 6 more fields]
data: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 6 more fields]
window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6efa422e
data: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 6 more fields]
data: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 6 more fields]
resultado: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 6 more fields]


Ejecuta la siguiente celda (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no.

Execute the following cell (don't modify its code) and it will tell you if your solution is correct or not. If its correct, it will show nothing, but if not it will show an error. More tests will be done, but this one indicates in a very reliable way if the implementation is correct or not.

In [71]:
assert(result.count() == 26)
assert(result.orderBy("origin_contract_number").collect()(24)(4) == "564367838")
assert(result.orderBy("origin_contract_number").collect()(19)(5) == "NULL")
assert(result.orderBy("origin_trade_number").collect()(16)(5) == "NULL")